In [ ]:
import polars as pl
import pycountry
import httpx

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from utils import numeric_iso_col, compress, decompress, year_month_col, fetch_acled_for_countries

In [108]:
pycountry.countries.get(alpha_3='ABW')

Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533')

In [109]:
kei = pl.read_csv('data/oecd_big_kei.csv')

In [110]:
num_kei = numeric_iso_col(kei, 'REF_AREA').drop_nulls('iso')

In [111]:
num_kei_keep = num_kei['iso', 'TIME_PERIOD', 'Measure', 'OBS_VALUE'].with_columns(
    pl.concat_str([pl.col('TIME_PERIOD'),
                   pl.col('iso')],
                   separator='-')
                   .alias('date_iso')
                   )

In [112]:
num_kei_keep

iso,TIME_PERIOD,Measure,OBS_VALUE,date_iso
str,str,str,f64,str
"""578""","""2024-04""","""Production volume""",2.764067,"""2024-04-578"""
"""578""","""2024-03""","""Production volume""",4.253215,"""2024-03-578"""
"""578""","""2020-09""","""Retail trade volume""",9.257266,"""2020-09-578"""
"""578""","""2020-08""","""Retail trade volume""",8.905579,"""2020-08-578"""
"""578""","""2020-07""","""Retail trade volume""",15.236052,"""2020-07-578"""
…,…,…,…,…
"""410""","""2019-07""","""Unemployment""",4.0,"""2019-07-410"""
"""410""","""2019-08""","""Unemployment""",3.6,"""2019-08-410"""
"""410""","""2019-09""","""Unemployment""",3.6,"""2019-09-410"""


In [113]:
num_kei_unique = num_kei_keep.unique()

In [116]:
num_kei_unique

iso,TIME_PERIOD,Measure,OBS_VALUE,date_iso
str,str,str,f64,str
"""372""","""2024-10""","""Merchandise exports""",22.116644,"""2024-10-372"""
"""724""","""2019-07""","""Composite leading indicator (C…",101.207393,"""2019-07-724"""
"""124""","""2022-02""","""Long-term interest rates""",1.879474,"""2022-02-124"""
"""792""","""2018-02""","""Producer prices""",1.486161,"""2018-02-792"""
"""756""","""2024-03""","""Production volume""",-0.184672,"""2024-03-756"""
…,…,…,…,…
"""578""","""2020-07""","""Production volume""",8.571429,"""2020-07-578"""
"""826""","""2021-11""","""M3""",8.505182,"""2021-11-826"""
"""250""","""2018-02""","""Retail trade volume""",109.16725,"""2018-02-250"""


In [141]:
num_kei_pivot = num_kei_unique.pivot(
    index=['iso', 'TIME_PERIOD'],
    on='Measure',
    values='OBS_VALUE',
    aggregate_function='first')

In [142]:
num_kei_pivot.shape

(3948, 24)

In [143]:
new_acled = fetch_acled_for_countries(num_kei_pivot)

Query to ACLED: {'year': '2020', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2023', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2019', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2024', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2021', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2022', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2018', 'iso': '710', 'page': '1'}
200
Query to ACLED: {'year': '2020', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2023', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2019', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2024', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2021', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2022', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2018', 'iso': '233', 'page': '1'}
200
Query to ACLED: {'year': '2020', 'iso': '428', 'page': '1'}
200
Query to ACLED: {'year': '2023', 'iso': 

In [ ]:
new_acled_iso_str = new_acled.with_columns(
    pl.col('iso').cast(pl.String).str.zfill(3))

In [ ]:
new_acled_iso_str.write_csv('data/inspected/acled_ocled_new.csv')

In [ ]:
#decompress('data/inspected/acled_oecd_countries_2018-2024.csv.xz')
#acled = pl.read_csv('data/inspected/acled_oecd_countries_2018-2024.csv', schema_overrides={'iso': pl.String})

In [156]:
acled_year_month = new_acled_iso_str.with_columns(
    pl.col('event_date')
    .str.to_date()
    .dt.strftime("%Y-%m")
    .alias('year_month'))
acled_grouped = acled_year_month.select(['year_month',
                                         'iso'])
acled_data = acled_grouped.group_by(['year_month',
                               'iso']).len().rename({'len':'incidents'})

In [157]:
acled_incidents = acled_data.select(['iso', 'year_month', 'incidents']).unique().sort(['iso', 'year_month'])

In [158]:
num_kei_year_month = num_kei_pivot.rename({'TIME_PERIOD': 'year_month'})
data = acled_incidents.join(num_kei_year_month, on=['year_month', 'iso'], how='left')

In [159]:
data

iso,year_month,incidents,Merchandise exports,Composite leading indicator (CLI),Long-term interest rates,Producer prices,Production volume,Composite consumer confidence,Short-term interest rates,Consumer prices,"Immediate interest rates, call money, interbank rate",Merchandise imports,Share prices,Retail trade volume,Permits issued for dwellings,Hourly earnings,M1,Passenger car registrations,M3,Employment,Nominal exchange rates,Composite business confidence,Unemployment,Reference series (GDP)
str,str,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""032""","""2018-01""",115,5.511756,null,null,null,null,null,null,1.757354,null,-3.218051,null,null,null,null,null,null,null,null,19.018799,null,null,null
"""032""","""2018-02""",131,-4.788811,null,null,null,null,null,null,25.437113,null,6.010661,null,null,null,null,null,null,null,null,19.839243,null,null,null
"""032""","""2018-03""",231,13.607041,null,null,null,null,null,null,2.341063,null,6.178861,null,null,null,null,null,null,null,null,20.230331,null,null,null
"""032""","""2018-04""",247,-4.389221,null,null,null,null,null,null,2.739032,null,17.023721,null,null,null,null,null,null,null,null,20.241778,null,null,null
"""032""","""2018-05""",193,-10.429283,null,null,null,null,null,null,2.075375,null,-5.886651,null,null,null,null,null,null,null,null,23.615701,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""840""","""2024-08""",809,176.715,99.708014,3.87,null,0.207929,73.056577,5.12,0.081389,5.33,-1.44129,1.297161,-0.544498,null,5.324773,0.30105,39.157867,175.722216,161425.0,null,-5.0,4.2,100.421483
"""840""","""2024-09""",873,-1.570891,99.826562,3.72,null,-0.619829,75.423653,4.86,2.440633,4.83,3.01785,3.080177,116.228249,null,5.26118,600.820649,2.61194,2.665058,161802.0,null,-5.0,4.1,100.388288
"""840""","""2024-10""",1052,-1.636486,99.970774,4.1,null,94.588909,75.85403,4.62,0.115128,4.83,3.707949,29.208569,0.374266,null,141.061183,0.343673,0.694215,176.964714,161456.0,null,-6.2,4.1,100.317277


In [160]:
X = data.drop(['iso', 'year_month', 'incidents'])
y = data['incidents']

In [9]:
data.write_csv('data/cleaned/data.csv')
compress('data/cleaned/data.csv')

PosixPath('data/cleaned/data.csv.xz')

### Global Acute Food Insecurity
* https://data.humdata.org/dataset/global-acute-food-insecurity-country-data
* Only covers developing nations, not compatible with current OECD countries

In [10]:
#ipc = pl.read_csv('data/inspected/ipc_global_national_long_latest_food_insecurity.csv')

### Global Mobility Report (Google):
* https://data.humdata.org/dataset/google-mobility-report
* Only covers 2020-2022

In [11]:
#global_mobility_report = pl.read_csv('data/inspected/Global_Mobility_Report.csv')

In [3]:
nightsky = pl.read_csv('data/ntl_adm2_monthly_2016.csv', null_values='NA')

In [7]:
nightsky

ISO_A3,ADM1CD_c,ADM2CD_c,NAM_0,NAM_1,NAM_2,date,n_gasflaring_locs,ntl_quality_prop_na,ntl_quality_prop_0_good,ntl_quality_prop_1_poor,ntl_quality_prop_2_gapfilled,ntl_sum,ntl_mean,ntl_median,ntl_max,ntl_q05,ntl_q95,ntl_gf_5km_sum,ntl_gf_5km_mean,ntl_gf_5km_median,ntl_gf_5km_max,ntl_gf_5km_q05,ntl_gf_5km_q95,ntl_nogf_5km_sum,ntl_nogf_5km_mean,ntl_nogf_5km_median,ntl_nogf_5km_max,ntl_nogf_5km_q05,ntl_nogf_5km_q95,ntl_gf_10km_sum,ntl_gf_10km_mean,ntl_gf_10km_median,ntl_gf_10km_max,ntl_gf_10km_q05,ntl_gf_10km_q95,ntl_nogf_10km_sum,ntl_nogf_10km_mean,ntl_nogf_10km_median,ntl_nogf_10km_max,ntl_nogf_10km_q05,ntl_nogf_10km_q95
str,str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-01-01""",0,0.0,0.874351,0.125649,0.0,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-02-01""",0,0.0,0.906542,0.09242,0.001038,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-03-01""",0,0.0,0.877466,0.122534,0.0,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-04-01""",0,0.0,0.0,1.0,0.0,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-05-01""",0,0.0,0.0,0.876428,0.123572,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZWE""","""ZWE008""","""ZWE008003""","""Zimbabwe""","""Matabeleland North""","""Hwange""","""2016-12-01""",0,0.000047,0.0,0.683857,0.316095,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152
"""ZWE""","""ZWE009""","""ZWE009005""","""Zimbabwe""","""Matabeleland South""","""Matobo""","""2016-12-01""",0,0.0,0.000108,0.96191,0.037982,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225
"""ZWE""","""ZWE009""","""ZWE009007""","""Zimbabwe""","""Matabeleland South""","""Bulilima""","""2016-12-01""",0,0.0,0.000526,0.987076,0.012397,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,

In [4]:
nightsky_date = year_month_col(nightsky, 'date')

In [6]:
nightsky_date['ISO_A3', 'ntl_sum', 'year_month'].group_by('ISO_A3', 'year_month').mean()

ISO_A3,year_month,ntl_sum
str,str,f64
"""BRB""","""2016-05""",927.857848
"""BRN""","""2016-09""",1757.215505
"""BTN""","""2016-10""",37.516818
"""BVT""","""2016-07""",1.92469
"""CCK""","""2016-03""",0.0
…,…,…
"""ZAF""","""2016-01""",29690.783609
"""ZAF""","""2016-05""",30393.661642
"""ZAF""","""2016-12""",30225.746505
